In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

import pandas as pd
import random 
import time
import numpy as np

API: http://selenium-python.readthedocs.io/api.html

Download chrome webdriver: https://sites.google.com/a/chromium.org/chromedriver/downloads

Chrome webdriver needs to sit in the same folder as this file. 

Other references: http://stackoverflow.com/questions/8900073/webdriver-screenshot

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
url = "https://www.asisonline.org/Membership/Member-Center/Pages/Member-Directory.aspx"

I am just going to let the human get to the main search reuslts page. 

login: 
jonathan.yu@umbocv.com	

password: 
33An]MDR9-~a_+6s

In [3]:
driver = webdriver.Chrome('./chromedriver', chrome_options=chrome_options)
driver.get(url)

# driver.close()

In [26]:
htmls = []

In [38]:
try: 
    driver.implicitly_wait(10)
    for x in range(0, 12): 
        listings = driver.find_elements_by_css_selector('div.psrch-Main')

        print("listings found: {0}".format(len(listings)))

        for listing in listings: 
            html = listing.get_attribute('innerHTML')
            htmls.append(html)
        next_button = driver.find_element_by_xpath('//*[@id="PageLinkNext"]')
        next_button.click()
except: 
    print("error")

listings found: 10
listings found: 10
listings found: 10
listings found: 10
listings found: 10
listings found: 10
listings found: 10
listings found: 10
listings found: 10
listings found: 3
error


In [14]:
len(htmls)

0

In [10]:
np.savetxt('../czech.txt', np.array(members),  delimiter=',', fmt="%s")

ValueError: setting an array element with a sequence

# The parsing of the text

In [39]:
soups = [BeautifulSoup(html, 'html.parser') for html in htmls]

members = []
for soup in soups: 
    members += soup.find_all('div', class_='psrch-results')

In [40]:
parsed_members = []

for member in members: 
    member_a = {}
    member_a['name'] = member.find('li', id='NameField').get_text()
    member_a['job_title'] = member.find('li', id='JobTitleField').get_text()
    member_a['company'] = member.find_all('li', id='JobTitleField')[1].get_text()
    member_a['city'] = member.find('li', id='LocationField').get_text().strip()
    if len(member.find_all('li', id='LocationField')) > 2:
        member_a['phone'] = member.find_all('li', id='LocationField')[2].get_text().strip()
    member_a['country'] = member.find('li', id='LocationField2').get_text().strip()
    member_a['email'] = member.find_all('a')[1].get_text()
    parsed_members.append(member_a)

In [41]:
pd.DataFrame(parsed_members).drop_duplicates().to_csv('../parsed_nordic.csv', index=None)